In [1]:
import os
os.environ['PATH'] += ':/opt/pbs/bin'
os.environ['PATH']

'/mnt/hpccs01/home/smitha50/src/dasktest/.venv/bin:/home/smitha50/.cargo/bin:/home/smitha50/.local/bin:/bin:/usr/bin:/home/smitha50/.fzf/bin:/opt/pbs/bin'

In [2]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores=4,
    memory="1GB",
    processes=1,
    walltime="00:20:00",
    resource_spec="select=1:ncpus=4:mem=1gb",
)

cluster.scale(jobs=20)

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.13.20.1:8787/status,
Dashboard: http://10.13.20.1:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.13.20.1:36787,Workers: 0
Dashboard: http://10.13.20.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
import dask.bag as db
import numpy as np


b = db.from_sequence(range(1000), npartitions=100)

def heavy_computation(x):
    arr = np.random.random((2000, 2000)) * x
    arr = np.linalg.svd(arr, compute_uv=False)  
    return arr.sum()

#mapped = b.map(lambda x: x ** x)
mapped = b.map(heavy_computation)

reduced = mapped.sum()

result = reduced.compute()

result

np.float64(11438422969.47559)

In [4]:
client.close()
cluster.close()